# Sesión 3.3 - Scrapy - Web noticias UM
En este ejemplo vamos a definir un Spider sencillo para obtener las noticias de la Universidad de Murcia que se encuentran en la URL https://www.um.es/web/sala-prensa/notas-de-prensa/historico?type=com.liferay.journal.model.JournalArticle&category=4753007

Para eso deberemos ver el código de la página y su estructura para encontrar la información que queremos extraer.

Una vez sabemos eso, podemos definir nuestra clase UMSpider.


In [ ]:
!pip3 install -U scrapy

In [2]:
import scrapy
import sys
import json
import locale
import time
import random
from bs4 import BeautifulSoup

# Para cada crawler que nos definimos nos debemos crear una clase Spider que debe heredar de la clase scrapy.Spider

class UMSpider (scrapy.Spider):
    name = 'unimurcia'

    # Decimos que el dominio válido es el de la UM
    allowed_domains = ['www.um.es']

    # podemos definir las páginas de inicio
    start_urls = ['https://www.um.es/web/sala-prensa/notas-de-prensa/historico?type=com.liferay.journal.model.JournalArticle&category=4753007']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    # debemos de implementar este método que se llamará para cada una de las páginas que se vayan analizando
    def parse (self, response):
        """
        @inherit

        @param self
        @param response
        """

        # Guardamos la URL del sitio que se está visitando
        url = str(response.request.url).strip()

        # Cogemos el contenido relevante y para eso debemos usar selectores CSS
        for article in response.css ('.NotaDePrensa'):
            # Cogemos el contenido del título
            title = str (article.css ('h1::text').get ()).strip()
            title = BeautifulSoup (title, 'html.parser').get_text().strip()
            # Cogemos el contenido del contenido
            content = "".join (article.css ('.mt-5 p').getall())
            content = BeautifulSoup (content, 'html.parser').get_text().strip().replace("\"","")

            # Cogemos también la fecha si existe
            date = str(article.css ('#NP-fecha').get().strip())
            if (date):
                date = BeautifulSoup (date, 'html.parser').get_text().strip()

            print ("-------------------------")
            print (url)
            print (title)
            print (content)
            print (date)
            print ("-------------------------")

            data = {
                'url' : url,
                'title': title,
                'content': content,
                'date': date
            }

            filename = str(random.random()).replace(".","") + ".json"

            # Guardamos el documento si tiene contenido y título
            if content and title:
                with open ('umu/' + filename, 'w') as f:
                    json.dump (data, f, indent = 4)


        # Obtenemos todas las otros links de la página representados por la etiqueta <a>
        url_in_current_document = response.css ('a')
        for next_page in url_in_current_document:
            # Para limitar que solamente se parseen las noticias dentro de 'https://www.um.es/web/sala-prensa/'
            # obtenemos el atributo href de la etiqueta <a> y parseamos la página
            url = str(next_page.css("a::attr(href)").get())
            if "sala-prensa" in url:
                yield response.follow (next_page, self.parse)


## Apartado 1.2
Para poder lanzar el Spider necesitamos que ejecutar el siguiente código donde se configuará y lanzará el proceso.
Hay que hacer notar que solamente se puede lanzar un proceso por cada sesión en Jupyter notebook es por eso por lo que se recomienda exportar el código en un script de Python .py para poder ejecutarlo desde la línea de comandos.

In [ ]:
import scrapy, os
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if not os.path.exists('umu'):
    os.mkdir('umu')

# Creamos el proceso con el RSSSpider
process.crawl(UMSpider)
# Ejecutamos el Crawler
process.start()

-------------------------
https://www.um.es/web/sala-prensa/-/42-letrados-completan-su-formaci%C3%B3n-en-liderazgo-judicial-por-la-escuela-de-pr%C3%A1ctica-jur%C3%ADdica/1.0?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico%3Ftype%3Dcom.liferay.journal.model.JournalArticle%26category%3D4753007
42 letrados completan su formación en liderazgo judicial por la Escuela de Práctica Jurídica
El rector de la Universidad de Murcia, José Luján, y el presidente del Tribunal Superior de Justicia de la Región de Murcia, Miguel Pasqual del Riquelme, han clausurado el I Programa Integral de Liderazgo Judicial para Letrados de la Administración de Justicia organizado por la Secretaría de Gobierno del TSJMU y la Escuela de Práctica Jurídica de la Universidad de Murcia, en colaboración con la Fundación Cajamurcia. La entrega de diplomas tuvo lugar en la sala de vistas de la Facultad de Derecho de la UMU.
En concreto, más de 40 integrantes del Cuerpo Jurídico Superior de letrados de la Administ